# RDD

Сгруппировать по ключу, просуммировать значения, вывести результат

In [1]:
%pyspark

rdd = sc.parallelize([(1,2), (3,4), (3,6), (4,5), (3, 4), (1, 5), (4, 1)])

result = rdd.groupByKey()\
            .map(lambda x:(x[0], sum(x[1])) )\
            .collect()

for (k, v) in result:
    print(f'For key {k} sum of values {v}')

Посчитать частоту встречаемости слов

In [2]:
%pyspark

lines = sc.parallelize([
    "a ab abc",
    "a ac abc",
    "b b ab abc"
    ])

counts = lines.flatMap(lambda x: x.split(' ')) \
              .map(lambda x: (x, 1))\
              .reduceByKey(lambda x, y: x + y)

output = counts.collect()

for (word, count) in output:
    print("%s: %i" % (word, count))

# market.events

Добавить колонки category_1, category_2, category_3 с категориями различного уровня

In [4]:
%pyspark

from pyspark.sql import functions as F

market_events = spark.table("market.events")
df_with_category = market_events.withColumn("category_1", F.when(market_events.category_code.isNotNull(), F.split("category_code", "\\.")[0]).otherwise(None)) \
                      .withColumn("category_2", F.when(market_events.category_code.isNotNull() & (F.size(F.split("category_code", "\\.")) > 1), 
                                                    F.split("category_code", "\\.")[1]).otherwise(None)) \
                      .withColumn("category_3", F.when(market_events.category_code.isNotNull() & (F.size(F.split("category_code", "\\.")) > 2), 
                                                    F.split("category_code", "\\.")[2]).otherwise(None))

df_with_category.show()

In [5]:
%pyspark
# Более простое решение:

market_events.withColumn("category_1", F.split("category_code", "\\.")[0]) \
             .withColumn("category_2", F.split("category_code", "\\.")[1]) \
             .withColumn("category_3", F.split("category_code", "\\.")[2]) \
             .show()

Вывести топ-3 брендов по количеству просмотров для каждой категории 2-го уровня

In [6]:
%pyspark

from pyspark.sql import functions as F
from pyspark.sql.window import Window

top_views_cat_2 = df_with_category.filter((df_with_category.event_type == 'view') & (df_with_category.brand.isNotNull())) \
                                  .groupBy('category_2', 'brand') \
                                  .agg(F.count("*").alias("views")) \
                                  .withColumn('rank', F.row_number().over(Window.partitionBy('category_2').orderBy(F.desc('views')))) \
                                  .filter(F.col('rank') <= 3) \
                                  .orderBy('category_2', 'rank')

top_views_cat_2.show()

# Датасет с треками

создание hw_3.tracks

In [8]:
%pyspark

import pyspark.sql.functions as f
from pyspark.sql.types import *

sch=ArrayType(StringType());

# важно что разделитель ', ' с пробелом, иначе пробелы добавятся в значения
tracks = spark.read.option("header", "true") \
        .option("escape", '"') \
        .option("InferSchema", "true") \
        .csv("/datasets/tracks.csv") \
        .withColumn("release_year", f.substring("release_date", 1, 4).cast(IntegerType())) \
        .withColumn("array_artist", f.split(f.regexp_replace(f.col("artists"), "[\]\[\']", ""),", ")) \
        .cache() #выделяем год в отдельную колонку и преобразуем колонку с артистами в массив

tracks_exp = tracks.select(  
                            "name", 
                            "popularity",
                            "danceability",
                            "energy",
                            "speechiness",
                            "acousticness",
                            "liveness",
                            "valence",
                            "release_year",
                            "artists",
                            f.explode(f.col("array_artist") ).alias("name_artist")
                        ) #создаем отдельную таблицу с развернутым массивом артистов
                        
tracks_exp.printSchema()

spark.sql("create database hw_3")
tracks_exp.write.mode("overwrite").saveAsTable("hw_3.tracks")

In [9]:
%pyspark

tracks = spark.table("hw_3.tracks")
z.show(tracks)

Какие артисты выпустили наибольшее число песен из годового топ-100 (по популярности)?

In [10]:
%pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

tracks = spark.table("hw_3.tracks")

tracks = tracks.dropDuplicates()

window_years = Window.partitionBy("release_year").orderBy(F.desc("popularity"))

top_songs_count_artists = tracks.withColumn("rank", F.row_number().over(window_years)) \
                                .filter(F.col("rank") <= 100) \
                                .groupBy("name_artist") \
                                .count() \
                                .orderBy(F.desc("count"))

top_songs_count_artists.show()

Вывести топ артистов, которые чаще других попадали в годовой топ-100 песен по популярности?

In [11]:
%pyspark
   
window_years = Window.partitionBy("release_year").orderBy(F.desc("popularity"))

top_artists_count = tracks.withColumn("rank", F.row_number().over(window_years)) \
                          .filter(F.col("rank") <= 100) \
                          .groupBy("release_year", "name_artist") \
                          .agg(F.countDistinct(F.col("name_artist")).alias("name_artist_count")) \
                          .groupBy("name_artist") \
                          .agg(F.count("*").alias("count_artists")) \
                          .orderBy(F.desc("count_artists"))

top_artists_count.show()

Какие артисты дольше других несколько лет подряд держались в ежегодном топ-100 песен по популярности?

In [12]:
%pyspark

window_years = Window.partitionBy("release_year").orderBy(F.desc("popularity"))
window_artists = Window.partitionBy("name_artist").orderBy("release_year")

top_artists_longer = tracks.withColumn("rank", F.row_number().over(window_years)) \
                          .filter(F.col("rank") <= 100) \
                          .drop(F.col("rank")) \
                          .select(F.col("name_artist"), F.col("release_year")) \
                          .dropDuplicates() \
                          .withColumn("group", F.col("release_year") - F.row_number().over(window_artists)) \
                          .groupBy("name_artist", "group") \
                          .agg(F.count("*").alias("num_years")) \
                          .drop(F.col("group")) \
                          .orderBy(F.desc("num_years"))

top_artists_longer.show()

Решение с udf

In [13]:
%pyspark

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

window_years = Window.partitionBy("release_year").orderBy(F.desc("popularity"))

top_songs_by_year = tracks.withColumn("rank", F.row_number().over(window_years)) \
                          .filter(F.col("rank") <= 100) \
                          .drop(F.col("rank"))

# Подсчет количества лет подряд для каждого артиста
@udf(returnType=IntegerType())
def count_consecutive_years(release_years):
    consecutive_years = 0
    prev_year = None
    for year in sorted(release_years):
        if prev_year is not None and year == prev_year + 1:
            consecutive_years += 1
        prev_year = year
    return consecutive_years


top_artists_longer = top_songs_by_year.groupBy("name_artist") \
                                      .agg(F.collect_list("release_year").alias("release_years")) \
                                      .withColumn("max_period", count_consecutive_years(F.col("release_years"))) \
                                      .filter(F.col("max_period") >= 2) \
                                      .select(F.col("name_artist"), F.col("max_period")) \
                                      .orderBy(F.desc("max_period"))

top_artists_longer.show()

Через lag

In [14]:
%pyspark
# https://stackoverflow.com/questions/56384625/pyspark-cumulative-sum-with-reset-condition

window_years = Window.partitionBy("release_year").orderBy(F.desc("popularity"))
window_artists = Window.partitionBy("name_artist").orderBy("release_year")

top_artists_longer = tracks.withColumn("rank", F.row_number().over(window_years)) \
                           .filter(F.col("rank") <= 100) \
                           .withColumn("prev_release_year", F.lag(F.col("release_year")).over(window_artists)) \
                           .withColumn("consecutive_years", F.when(F.col("release_year") - F.col("prev_release_year") == 1, 1).otherwise(0)) \
                           .withColumn("period", F.sum("consecutive_years").over(window_artists)) \
                           .filter(F.col("period") >= 2) \
                           .groupBy(F.col("name_artist")) \
                           .agg(F.max("period").alias("max_period")) \
                           .orderBy(F.desc("max_period"))
top_artists_longer.show()                           

Для каждой аудиохарактеристики вывести топ 3 артистов по среднему значению каждой аудиохарактеристики его песен. Дополнительно: отнормировать на среднее значение аудиохарактеристики в год выхода песен.

In [15]:
%pyspark

window_danceability = Window.orderBy(F.desc("avg_danceability"))
window_acousticness = Window.orderBy(F.desc("avg_acousticness"))
window_energy = Window.orderBy(F.desc("avg_energy"))
window_speechiness = Window.orderBy(F.desc("avg_speechiness"))
window_liveness = Window.orderBy(F.desc("avg_liveness"))
window_valence = Window.orderBy(F.desc("avg_valence"))


avg_by_artist = tracks.groupBy("name_artist") \
                      .agg(F.avg("danceability").alias("avg_danceability"),
                           F.avg("acousticness").alias("avg_acousticness"),
                           F.avg("energy").alias("avg_energy"),
                           F.avg("speechiness").alias("avg_speechiness"),
                           F.avg("liveness").alias("avg_liveness"),
                           F.avg("valence").alias("avg_valence"))

top_danceability = avg_by_artist.withColumn("rank", F.row_number().over(window_danceability)) \
                                .filter(F.col("rank") <= 3) \
                                .select(F.col("name_artist"), F.col("avg_danceability"))

top_acousticness = avg_by_artist.withColumn("rank", F.row_number().over(window_acousticness)) \
                                .filter(F.col("rank") <= 3) \
                                .select(F.col("name_artist"), F.col("avg_acousticness"))

top_energy = avg_by_artist.withColumn("rank", F.row_number().over(window_energy)) \
                          .filter(F.col("rank") <= 3) \
                          .select(F.col("name_artist"), F.col("avg_energy"))

top_speechiness = avg_by_artist.withColumn("rank", F.row_number().over(window_speechiness)) \
                               .filter(F.col("rank") <= 3) \
                               .select(F.col("name_artist"), F.col("avg_speechiness"))

top_liveness = avg_by_artist.withColumn("rank", F.row_number().over(window_liveness)) \
                            .filter(F.col("rank") <= 3) \
                            .select(F.col("name_artist"), F.col("avg_liveness"))

top_valence = avg_by_artist.withColumn("rank", F.row_number().over(window_valence)) \
                           .filter(F.col("rank") <= 3) \
                           .select(F.col("name_artist"), F.col("avg_valence"))
                           
top_danceability.show()
top_acousticness.show()
top_energy.show()
top_speechiness.show()
top_liveness.show()
top_valence.show()

In [16]:
%pyspark

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

    
avg_by_year = tracks.groupBy("release_year") \
                      .agg(F.avg("danceability").alias("avg_danceability"),
                           F.avg("acousticness").alias("avg_acousticness"),
                           F.avg("energy").alias("avg_energy"),
                           F.avg("speechiness").alias("avg_speechiness"),
                           F.avg("liveness").alias("avg_liveness"),
                           F.avg("valence").alias("avg_valence"))
                           
min_by_year = tracks.groupBy("release_year") \
                      .agg(F.min("danceability").alias("min_danceability"),
                           F.min("acousticness").alias("min_acousticness"),
                           F.min("energy").alias("min_energy"),
                           F.min("speechiness").alias("min_speechiness"),
                           F.min("liveness").alias("min_liveness"),
                           F.min("valence").alias("min_valence"))
                           
max_by_year = tracks.groupBy("release_year") \
                      .agg(F.max("danceability").alias("max_danceability"),
                           F.max("acousticness").alias("max_acousticness"),
                           F.max("energy").alias("max_energy"),
                           F.max("speechiness").alias("max_speechiness"),
                           F.max("liveness").alias("max_liveness"),
                           F.max("valence").alias("max_valence"))
                           
df_with_agg = tracks.join(F.broadcast(avg_by_year), "release_year") \
                    .join(F.broadcast(min_by_year), "release_year") \
                    .join(F.broadcast(max_by_year), "release_year")
                    
                                            
normalized_df = df_with_agg.withColumn("norm_danceability", (F.col("danceability") - F.col("avg_danceability")) / (F.col("max_danceability") - F.col("min_danceability"))) \
                           .withColumn("norm_acousticness", (F.col("acousticness") - F.col("avg_acousticness")) / (F.col("max_acousticness") - F.col("min_acousticness"))) \
                           .withColumn("norm_energy", (F.col("energy") - F.col("avg_energy")) / (F.col("max_energy") - F.col("min_energy"))) \
                           .withColumn("norm_speechiness", (F.col("speechiness") - F.col("avg_speechiness")) / (F.col("max_speechiness") - F.col("min_speechiness"))) \
                           .withColumn("norm_liveness", (F.col("liveness") - F.col("avg_liveness")) / (F.col("max_liveness") - F.col("min_liveness"))) \
                           .withColumn("norm_valence", (F.col("valence") - F.col("avg_valence")) / (F.col("max_valence") - F.col("min_valence"))) \
                           .select(F.col("release_year"), F.col("norm_danceability"), F.col("norm_acousticness"), F.col("norm_energy"), F.col("norm_speechiness"), F.col("norm_liveness"), F.col("norm_valence")) \
                           #.orderBy(F.col("release_year"))

normalized_df.show()